In [ ]:
%%capture
%config Completer.use_jedi = False
%config InlineBackend.figure_formats = ['svg']
import os

STATIC_WEB_PAGE = {"EXECUTE_NB", "READTHEDOCS"}.intersection(os.environ)

# Install on Google Colab
import subprocess
import sys

from IPython import get_ipython

install_packages = "google.colab" in str(get_ipython())
if install_packages:
    for package in ["qrules[doc]", "graphviz"]:
        subprocess.check_call(
            [sys.executable, "-m", "pip", "install", package]
        )

# Custom topologies

As illustrated in {doc}`/usage/reaction`, the {class}`.StateTransitionManager` offers you a bit more flexibility than the façade function {func}`.generate_transitions` used in the main {doc}`/usage` page. In this notebook, we go one step further, by specifying a custom {class}`.Topology` via {attr}`.StateTransitionManager.topologies`.

In [ ]:
import graphviz

import qrules
from qrules import InteractionType, StateTransitionManager
from qrules.topology import Edge, Topology

## 2-to-2 topology

As a simple example, we start with a 2-to-2 scattering topology. We define it as follows:

:::{margin}

We use the fact that {class}`.Topology` can be constructed with {class}`~typing.Iterable`s, like the ones created with {obj}`range` and {func}`enumerate`.

:::

In [ ]:
topology = Topology(
    nodes=range(2),
    edges=enumerate(
        [
            Edge(None, 0),
            Edge(None, 0),
            Edge(1, None),
            Edge(1, None),
            Edge(0, 1),
        ],
        -2,
    ),
)

In [ ]:
dot = qrules.io.asdot(
    topology,
    render_resonance_id=True,
    render_node=True,
    render_initial_state_id=True,
)
graphviz.Source(dot)

First, we construct a {class}`.StateTransitionManager` for the transition $K^-K^+ \to \pi^+\pi^-$. The constructed {class}`.Topology` can then be inserted via its {attr}`~.StateTransitionManager.topologies` attribute:

In [ ]:
stm = StateTransitionManager(
    initial_state=["K-", "K+"],
    final_state=["pi-", "pi+"],
    formalism="canonical",
)
stm.set_allowed_interaction_types([InteractionType.STRONG, InteractionType.EM])
stm.topologies = (topology,)  # tuple is immutable

For the rest, the process is just the same as in {doc}`/usage/reaction`:

In [ ]:
problem_sets = stm.create_problem_sets()
reaction_kk = stm.find_solutions(problem_sets)

In [ ]:
dot = qrules.io.asdot(reaction_kk, collapse_graphs=True)
graphviz.Source(dot)

:::{warning}

It is not yet possible to give the initial state a certain energy. So some collider process like $e^-e^+\to\pi^+\pi$ does not result in a large number of resonances.

:::

In [ ]:
stm.initial_state = ["e-", "e+"]
problem_sets = stm.create_problem_sets()
reaction_ep = stm.find_solutions(problem_sets)

In [ ]:
dot = qrules.io.asdot(reaction_ep, collapse_graphs=True)
graphviz.Source(dot)

What can do at most, is switch off {class}`.MassConservation`, either through the constructor of the {class}`.StateTransitionManager`, or by modifying {class}`.ProblemSet`.

In [ ]:
stm = StateTransitionManager(
    initial_state=["e-", "e+"],
    final_state=["pi-", "pi+"],
    formalism="canonical",
    mass_conservation_factor=None,
)
stm.set_allowed_interaction_types([InteractionType.STRONG, InteractionType.EM])
stm.topologies = [topology]
problem_sets = stm.create_problem_sets()
reaction_ep_no_mass = stm.find_solutions(problem_sets)

In [ ]:
dot = qrules.io.asdot(reaction_ep_no_mass, collapse_graphs=True)
graphviz.Source(dot)